![puma logo](https://github.com/nasa/puma/raw/main/doc/source/puma_logo.png)
# Welcome to the PuMA tutorial

The objective of this notebook is to familiarize new users with the main datastructures that stand at the basis of the PuMA project, and outline the functions to compute material properties (please refer to these papers ([1](https://www.sciencedirect.com/science/article/pii/S2352711018300281), [2](https://www.sciencedirect.com/science/article/pii/S235271102100090X)) for more details on the software).

# Installation setup and imports

The first code block will execute the necessary installation and package import. 

If you are running this jupyter notebook locally on your machine, assuming you have already installed the software, then the installation step will be skipped.


In [ ]:
if 'google.colab' in str(get_ipython()):
    !pip install 'git+https://github.com/nasa/puma'
    !pip install -q piglet pyvirtualdisplay
    !apt-get -qq install xvfb

import numpy as np
import pumapy as puma
import pyvista as pv
import scipy.ndimage as nd
import os
import sys

if 'google.colab' in str(get_ipython()):
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(600, 400))
    display.start()  # necessary for pyvista interactive plots
    
else:  # NORMAL JUPYTER NOTEBOOK
    # for interactive slicer (only static allowed on Colab)
    %matplotlib widget

## Tutorial: Elasticity
In this tutorial we demonstrate the use of the compute_elasticity and compute_stress_analysis functions. These functions rely on a stress analysis solver that uses the finite volume Multi-Point Stress Approximation (MPSA) method.

We will run four different verification cases. Change the path of the file outputs:

In [ ]:
export_path = "out"  # CHANGE THIS PATH

if not os.path.exists(export_path):
    os.makedirs(export_path)

### Example 1: harmonic averaging, in series along x with free sides

The first example that we run is for a block of material split into two phases with different properties.

In [ ]:
export_name = 'halfmat'
X = 20
Y = 20
Z = 20
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws[int(X / 2):] = 2
# ws.show_matrix()

puma.render_volume(ws, solid_color=(255,255,255), notebook=True, style='edges', cmap='jet')

We can now assign the elasticity of the two materials and compute the resulting overall elasticity of the two phases combined as follows:

In [ ]:
elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)
elast_map.add_isotropic_material((2, 2), 400, 0.1)

 In this example, we use the compute_elasticity function, which is useful specifically to compute the homogenized (or effective) elasticity of a multi-phase material. This function imposes a unit displacement along the direction specified by holding the last slice of voxels in place with dirichlet boundary conditions. The side boundary conditions can be set as either 'p'eriodic, 's'ymmetric or 'f'ree. In this case we set them as free with 'f'.

In [ ]:
C, u, s, t = puma.compute_elasticity(ws, elast_map, direction='x', side_bc='f', solver_type="direct")
print("Elasticity tensor first column:")
print(C[0])
print(C[1])
print(C[2])

Now we can visualize the displacement and stress fields as:

In [ ]:
results = puma.Workspace()
results.orientation = u[:, :Y//2]  # cut domain in half to show internal stresses
scale_factor = 10

p = pv.Plotter(shape=(2, 3))
p.subplot(0, 0)
p.add_text("Colored by sigma_xx")
results.matrix = s[:, :Y//2, :, 0]  # assign direct stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 1)
p.add_text("Colored by sigma_yy")
results.matrix = s[:, :Y//2, :, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 2)
p.add_text("Colored by sigma_zz")
results.matrix = s[:, :Y//2, :, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 0)
p.add_text("Colored by tau_yz")
results.matrix = t[:, :Y//2, :, 0]  # assign shear stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 1)
p.add_text("Colored by tau_xz")
results.matrix = t[:, :Y//2, :, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 2)
p.add_text("Colored by tau_yz")
results.matrix = t[:, :Y//2, :, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.show()

Finally, we can export all of this data (domain, displacement, direct stress, shear stress) in a single .vti file as: 

In [ ]:
puma.export_vti(os.path.join(export_path, export_name), {"ws": ws, "disp": u, "sigma": s, "tau": t})

### Example 2: full built-in beam

The second case is for a fully built-in homogeneous beam with a z displacement in the middle. Because of the symmetry of this case, we only model half of it.

In [ ]:
export_name = 'builtinbeam'
X = 10
Y = 50
Z = 10
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

puma.render_volume(ws, cutoff=(0, 255), solid_color=(255,255,255), style='edges', notebook=True)

We then run set its elasticity as:

In [ ]:
elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

Since we want to set a specific displacement, we need to have more control on the type of boundary conditions we set. This can be done by creating an ElasticityBC object as:

In [ ]:
bc = puma.ElasticityBC(ws)
bc.dirichlet[:, 0] = 0  # dirichlet displacement to zero on the y -ve face (i.e. hold in place)
bc.dirichlet[:, -1, :, :2] = 0  # dirichlet y and z displacements on y +ve face (i.e. free slip in x)
bc.dirichlet[:, -1, :, 2] = -1  # dirichlet z displacement of -1 on y +ve face
# puma.Workspace.show_orientation(bc)

# Plot the boundary conditions array inside the ElasticityBC object
dir_copy = bc.dirichlet.copy()
# the unset DOF are usually set to Inf, but for plotting purposes we set them to NaN
dir_copy[np.isinf(dir_copy)] = np.NaN
p = pv.Plotter(shape=(1, 3))
p.subplot(0, 0)
p.add_text("Dirichlet displacement in x")
puma.render_volume(dir_copy[:,:,:,0], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 1)
p.add_text("Dirichlet displacement in y")
puma.render_volume(dir_copy[:,:,:,1], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.subplot(0, 2)
p.add_text("Dirichlet displacement in z")
puma.render_volume(dir_copy[:,:,:,2], notebook=True, add_to_plot=p, plot_directly=False, cmap='jet')
p.show()

In [ ]:
u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")

In [ ]:
results = puma.Workspace()
results.orientation = u[:X//2]
scale_factor = 10

p = pv.Plotter(shape=(2, 3))
p.subplot(0, 0)
p.add_text("Colored by sigma_xx")
results.matrix = s[:X//2, :, :, 0]  # assign direct stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 1)
p.add_text("Colored by sigma_yy")
results.matrix = s[:X//2, :, :, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 2)
p.add_text("Colored by sigma_zz")
results.matrix = s[:X//2, :, :, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 0)
p.add_text("Colored by tau_yz")
results.matrix = t[:X//2, :, :, 0]  # assign shear stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 1)
p.add_text("Colored by tau_xz")
results.matrix = t[:X//2, :, :, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 2)
p.add_text("Colored by tau_yz")
results.matrix = t[:X//2, :, :, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.show()

In [ ]:
puma.export_vti(os.path.join(export_path, export_name), {"ws": ws, "disp": u, "sigma": s, "tau": t})

### Example 3: plate with a hole

In this example, we model a plate with a hole in the middle pulled in the x direction by a certain displacement.

In [ ]:
export_name = 'platehole'
X = 50
Y = 50
Z = 3
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

# creating circular hole
mask = np.ones((X, Y, Z), dtype=bool)
mask[X//2, Y//2] = 0
distance_mask = nd.morphology.distance_transform_edt(mask)
max_distance = np.max(distance_mask)
distance_mask_display = (distance_mask*255./max_distance).astype(dtype=np.uint8)
in_range = distance_mask <= 17  # this sets how big is the hole with a threshold
ws[in_range] = 0

# setting material
elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

# setting dirichlet boundary conditions
bc = puma.ElasticityBC(ws)
bc.dirichlet[0, :, :, 0] = 0
bc.dirichlet[-1, :, :, 0] = 1

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")

In [ ]:
results = puma.Workspace()
u[ws.matrix == 0] = np.NAN  # set air displacement to NAN to avoid plotting it
results.orientation = u[:, :, :Z//2]
scale_factor = 10

p = pv.Plotter(shape=(2, 3))
p.subplot(0, 0)
p.add_text("Colored by sigma_xx")
results.matrix = s[:, :, :Z//2, 0]  # assign direct stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 1)
p.add_text("Colored by sigma_yy")
results.matrix = s[:, :, :Z//2, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 2)
p.add_text("Colored by sigma_zz")
results.matrix = s[:, :, :Z//2, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 0)
p.add_text("Colored by tau_yz")
results.matrix = t[:, :, :Z//2, 0]  # assign shear stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 1)
p.add_text("Colored by tau_xz")
results.matrix = t[:, :, :Z//2, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 2)
p.add_text("Colored by tau_yz")
results.matrix = t[:, :, :Z//2, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.show(cpos="xy")

In [ ]:
puma.export_vti(os.path.join(export_path, export_name), {"ws": ws, "disp": u, "sigma": s, "tau": t})

### Example 4: cracked plate

In this final example, we model a plate with a single row of voxels removed, mimicking a crack. 

In [ ]:
export_name = 'crackedplate'
X = 25
Y = 100
Z = 3
ws = puma.Workspace.from_shape_value((X, Y, Z), 1)
ws.voxel_length = 1

ws[:10, Y//2-1:Y//2+1] = 0

elast_map = puma.ElasticityMap()
elast_map.add_isotropic_material((1, 1), 200, 0.3)

bc = puma.ElasticityBC(ws)
bc.dirichlet[:, 0, :, 1] = 0
bc.dirichlet[:, -1, :, 1] = 1

u, s, t = puma.compute_stress_analysis(ws, elast_map, bc, side_bc='f', solver_type="direct")

In [ ]:
results = puma.Workspace()
u[ws.matrix == 0] = np.NAN  # set air displacement to NAN to avoid plotting it
results.orientation = u[:, :, :Z//2]
scale_factor = 10

p = pv.Plotter(shape=(2, 3))
p.subplot(0, 0)
p.add_text("Colored by sigma_xx")
results.matrix = s[:, :, :Z//2, 0]  # assign direct stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 1)
p.add_text("Colored by sigma_yy")
results.matrix = s[:, :, :Z//2, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(0, 2)
p.add_text("Colored by sigma_zz")
results.matrix = s[:, :, :Z//2, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 0)
p.add_text("Colored by tau_yz")
results.matrix = t[:, :, :Z//2, 0]  # assign shear stresses to matrix
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 1)
p.add_text("Colored by tau_xz")
results.matrix = t[:, :, :Z//2, 1]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.subplot(1, 2)
p.add_text("Colored by tau_yz")
results.matrix = t[:, :, :Z//2, 2]
puma.render_warp(results, color_by='matrix', scale_factor=scale_factor, style='edges', notebook=True, add_to_plot=p, plot_directly=False)
p.show(cpos="xy")

In [ ]:
puma.export_vti(os.path.join(export_path, export_name), {"ws": ws, "disp": u, "sigma": s, "tau": t})